# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [24]:
#!pip install chromadb langchain langchain_community langchain_openai

In [25]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [26]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [27]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [28]:
llm = OpenAI(temperature=0)

In [29]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_edu.txt")

In [30]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-edu")

In [31]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [32]:
from langchain_community.document_loaders import WebBaseLoader

In [33]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [34]:
docs = loader.load()

# Use RecursiveCharacterTextSplitter for better handling of web content
# It tries multiple separators and is more intelligent about splitting
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # Try these separators in order
)

ruff_texts = recursive_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [35]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [36]:
tools = [
    Tool(
        name="State of Education QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the education address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [37]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.invoke(
    "What specific solutions or initiatives does the speaker propose to address the identified challenges, and what role do technologies like AI play?"
)



> Entering new AgentExecutor chain...
 The speaker is discussing solutions and initiatives, so I should use the State of Education QA System.
Action: State of Education QA System
Action Input: "What solutions or initiatives are proposed to address the challenges in education? What role does AI play?" The speaker is discussing solutions and initiatives, so I should use the State of Education QA System.
Action: State of Education QA System
Action Input: "What solutions or initiatives are proposed to address the challenges in education? What role does AI play?"
Observation:  The American Rescue Plan provides funding for schools to hire teachers and help students catch up on lost learning. Additionally, individuals are encouraged to sign up as tutors or mentors to support students. AI could potentially play a role in providing personalized learning and support for students.
Thought:
Observation:  The American Rescue Plan provides funding for schools to hire teachers and help students cat

In [ ]:
agent.invoke("How does the speaker characterize the current state of the 'educational union,' and what specific areas of progress are highlighted?")

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [ ]:
tools = [
    Tool(
        name="State of Education QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the education address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [ ]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.invoke(
    "What specific solutions or initiatives does the speaker propose to address the identified challenges, and what role do technologies like AI play?"
)

In [ ]:
agent.invoke("What are the key challenges or pain points in education that the speaker identifies, particularly concerning students and parents?")

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [ ]:
tools = [
    Tool(
        name="State of Education QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the education address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [ ]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.invoke(
    "Beyond academic achievement, what broader goals or values does the speaker emphasize as crucial for the future of education?"
)